# Analisis de propeidades estaticas
- Energía cinética media -> Temperatura media
- Fuerza cuadratica media
- Presión
- ...

## Importación de librerías y lectura de datos

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
codigos=["110-160","160-210","110-160","210-260","260-310","310-360","360-410","410-460","460-510","510-560"] #Tiempo de inicio de simulacion-Tiempo de fin de simulacion

DFS_vx=[]
DFS_vy=[]
DFS_vz=[]

DATOS=[]

for codigo in codigos:
    DFS_vx.append(pd.read_csv("../resultados/dinamica/Simulacion_"+ codigo + "/velocidad_x.txt", sep=r"\s+", header=None))
    DFS_vy.append(pd.read_csv("../resultados/dinamica/Simulacion_"+ codigo + "/velocidad_y.txt", sep=r"\s+", header=None))
    DFS_vz.append(pd.read_csv("../resultados/dinamica/Simulacion_"+ codigo + "/velocidad_z.txt", sep=r"\s+", header=None))
    DATOS.append(pd.read_csv("../resultados/dinamica/Simulacion_"+ codigo + "/gname2.txt",skiprows=1, sep=r"\s+", header=None, names=['t','Ec','Ep','E_tot','W','T','P_corr','F2']))

In [18]:
tiempos=[df.t.to_numpy() for df in DATOS]

tiempos[0]

array([1.000e-02, 2.000e-02, 3.000e-02, ..., 4.998e+01, 4.999e+01,
       5.000e+01], shape=(5000,))

### Formato de datos importados

- En las columnas tenemos las particulas de la caja etiquetadas del 0 al 499
- En las filas tenemos instantes sucesivos de tiempo grabados de nuestra simulacion.

In [21]:
DATOS[-1].iloc[:,1:].agg(["mean","std"])
DATOS[-1].max()

t           50.000000
Ec        1089.322000
Ep       -1630.208000
E_tot     -664.940600
W          -69.628890
T            1.452430
P_corr       0.594222
F2         960.669600
dtype: float64

In [22]:
F2s_mean=[]
F2s_std=[]

Ecs_mean=[]
Ecs_std=[]

Ps_mean=[]
Ps_std=[]


for df in DATOS:
    F2s_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","F2"])
    F2s_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","F2"])

    Ecs_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","Ec"])
    Ecs_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","Ec"])

    Ps_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","P_corr"])
    Ps_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","P_corr"])


F2s_mean=np.array(F2s_mean)
F2s_std=np.array(F2s_mean)

Ecs_mean=np.array(Ecs_mean)
Ecs_std=np.array(Ecs_std)

Ps_mean=np.array(Ps_mean)
Ps_std=np.array(Ps_std)


"""
print("Ec=",)

print(F2s_mean)
print(F2s_std)
print(Ecs_mean)
print(Ecs_std)
print(Ps_mean)
print(Ps_std)

"""

# Media global = media de las medias
F2_mean = np.mean(F2s_mean)
Ec_mean = np.mean(Ecs_mean)
P_mean  = np.mean(Ps_mean)

# Error estándar de la media global --> ¿Deberia tener en cuenta las std de cada bloque simulado?
F2_err = np.std(F2s_mean, ddof=1) / np.sqrt(len(F2s_mean))
Ec_err = np.std(Ecs_mean, ddof=1) / np.sqrt(len(Ecs_mean))
P_err  = np.std(Ps_mean, ddof=1) / np.sqrt(len(Ps_mean))

print(f"<F²> = {F2_mean:.6f} ± {F2_err:.6f}")
print(f"<Ec> = {Ec_mean:.6f} ± {Ec_err:.6f}")
print(f"<P>  = {P_mean:.6f} ± {P_err:.6f}")

<F²> = 326.687683 ± 56.087657
<Ec> = 1023.630884 ± 0.694273
<P>  = 0.232922 ± 0.002128
